Monting google drive, use this if you want to run the file with google colab and save the data in drive.

In [ ]:
# before doing this , copy the CSV files into a folder "data" in your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libs

In [ ]:
!pip install surprise #installing it because it is not installed by default
from surprise import AlgoBase
from surprise import PredictionImpossible
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise import KNNBasic
from surprise import NMF
from surprise import accuracy
from numpy import random
import os
import csv
import sys
from collections import defaultdict
from operator import itemgetter
import numpy as np
import heapq
import math
import itertools
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise.model_selection import train_test_split


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095442 sha256=80dae6f5285f71e7269559f3f529b6d6a802feb106cf8c15ea5582a05e63c909
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


Data provider class, change the ratings path and books path according to your configuration.

In [ ]:
class DataProvider:
    bookID_to_name = {}
    name_to_BookID = {}
    ratingsPath = '/content/drive/MyDrive/data/5K_users_ratings.csv'
    booksPath = '/content/drive/MyDrive/data/books_data.csv'

    def loadData(self):

        # Look for files relative to the directory we are running from
        os.chdir(os.path.dirname(sys.argv[0]))

        ratingsDataset = 0
        self.bookID_to_name = {}
        self.name_to_BookID = {}

        reader = Reader( sep=',', skip_lines=1)

        ratingsDataset = Dataset.load_from_file(self.ratingsPath, reader=reader)

        with open(self.booksPath, newline='', encoding='UTF-8') as csvfile:
            bookReader = csv.reader(csvfile)
            next(bookReader)  # Skip header line
            for row in bookReader:
                bookID = int(row[0])
                bookName = row[1]
                self.bookID_to_name[bookID] = bookName
                self.name_to_BookID[bookName] = bookID
        print(ratingsDataset)
        return ratingsDataset

    def getUserRatings(self, user):
        userRatings = []
        with open(self.ratingsPath, newline='') as csvfile:
            ratingReader = csv.reader(csvfile)
            next(ratingReader)
            for row in ratingReader:
                userID = int(row[0])
                if (user == userID):
                    book_id = int(row[1])
                    rating = float(row[2])
                    userRatings.append((book_id, rating))

        return userRatings

    def getPopularityRanks(self):
        ratings = defaultdict(int)
        rankings = defaultdict(int)
        with open(self.ratingsPath, newline='') as csvfile:
            ratingReader = csv.reader(csvfile)
            next(ratingReader)
            for row in ratingReader:
                bookID = int(row[1])
                ratings[bookID] += 1
        rank = 1
        for bookID, ratingCount in sorted(ratings.items(), key=lambda x: x[1], reverse=True):
            rankings[bookID] = rank
            rank += 1
        return rankings

    def getGenres(self):
        genres = defaultdict(list)
        genreIDs = {}
        maxGenreID = 0
        with open(self.booksPath, newline='', encoding='ISO-8859-1') as csvfile:
            bookReader = csv.reader(csvfile)
            next(bookReader)  # Skip header line
            for row in bookReader:
                bookID = int(row[0])
                genreList = [x.strip('[] ') for x in row[5].split(',')]
                genreIDList = []
                for genre in genreList:
                    if genre in genreIDs:
                        genreID = genreIDs[genre]
                    else:
                        genreID = maxGenreID
                        genreIDs[genre] = genreID
                        maxGenreID += 1
                    genreIDList.append(genreID)
                genres[bookID] = genreIDList
        # Convert integer-encoded genre lists to bitfields that we can treat as vectors
        for (bookID, genreIDList) in genres.items():
            bitfield = [0] * maxGenreID
            for genreID in genreIDList:
                bitfield[genreID] = 1
            genres[bookID] = bitfield
        return genres

    def getYears(self):
        years = defaultdict(int)
        with open(self.booksPath, newline='', encoding='ISO-8859-1') as csvfile:
            bookReader = csv.reader(csvfile)
            next(bookReader)
            for row in bookReader:
                bookID = int(row[0])
                year = int(row[3])
                if year:
                    years[bookID] = year
        return years

    def getStandardGeners(self):
        geners = defaultdict(list)
        with open(self.booksPath, newline='', encoding='ISO-8859-1') as csvfile:
            bookReader = csv.reader(csvfile)
            next(bookReader)
            for row in bookReader:
                bookID = int(row[0])
                bookGeners = row[5].split(',')
                for gener in bookGeners :
                  geners[bookID].append(gener)
        return geners

    def getAuthors(self):
        authors = defaultdict(list)
        with open(self.booksPath, newline='', encoding='ISO-8859-1') as csvfile:
            bookReader = csv.reader(csvfile)
            next(bookReader)
            for row in bookReader:
                bookID = int(row[0])
                authors[bookID]=[x.strip('[] ') for x in row[2].split(',')]
        return authors

    def getBookName(self, bookID):
        if bookID in self.bookID_to_name:
            return self.bookID_to_name[bookID]
        else:
            return ""

    def getBookID(self,bookName):
        if bookName in self.name_to_BookID:
            return self.name_to_BookID[bookName]
        else:
            return 0

CoAlgorithm class, based on KNNm algorithm.

In [ ]:
#we use the famous KNN algorithm as one of our recommenders
#we dont need to code the hole algo because its on ready exist in suprise lib as "KNNWithMeans"
class CoAlgorithm(KNNWithMeans):
  def __init__(self, k=40):
        #init the algorithm , we want the CF user-user not item-item
        KNNWithMeans.__init__(self,sim_options={'name': 'cosine','user_based': True})
        self.k = k

  #fitting the trainset into the algoerithm
  #the fit function of KNNWithMeans has on ready a fonction "compute similarity" built in
  #so we dont need to write one

  def fit(self, trainset):
        #fiting the data
        KNNWithMeans.fit(self, trainset)
        

  #every algorithm has a function called "pridect" 
  #that uses another function called "estimate" that generate the predicted rating      
  #the estimate function are on ready built in , so we dont need to define it in this algo
  #on other algorithms, we will define it


  #the predict function generate a rating between one user and one item
  #to predict all estimated ratings of a user i create this function
  def EstamedRatingsForUser(self,u):
        prid = []
        for item in range(self.trainset.n_items):
              
              rating = self.predict(str(u) , str(self.trainset.to_raw_iid(item)))
              prid.append(rating)

        return prid

CBAlgorithm class, a content-based Algorithm

In [ ]:
#extended from an empty recommender template called "AlgoBase"
class CBAlgorithm(AlgoBase):

    def __init__(self, k=40, sim_options={}):
        AlgoBase.__init__(self)
        self.k = k

    def fit(self, trainset , load= False):
        #fiting the data
        AlgoBase.fit(self, trainset)

        # Load up metadata for every book
        BD = DataProvider()
        genres = BD.getGenres()
        years = BD.getYears()
        authors = BD.getAuthors()

        #if you have a copy od similarities 
        if load :
          self.similarities = self.importSimilarities()
          print("...done.")
          return self

        #generating sim distance for every book combination as a 2x2 matrix
        self.similarities = np.zeros((self.trainset.n_items, self.trainset.n_items))

        # to calculate it
        # Compute item similarity matrix based on content attributes
        print("Computing content-based similarity matrix...")
        for thisItem in range(self.trainset.n_items):
            if (thisItem % 100 == 0):
                #just to print the progress every 100 books so you wouldnt suicide
                print(thisItem, " of ", self.trainset.n_items)

            for otherItem in range(thisItem+1, self.trainset.n_items):
                #get the inner IDs
                thisBookID = int(self.trainset.to_raw_iid(thisItem)) 
                otherBookID = int(self.trainset.to_raw_iid(otherItem))

                #get the sub similarities
                genreSimilarity = self.computeGenreSimilarity(thisBookID, otherBookID, genres)
                if genreSimilarity == 0 :
                  sim = 0 
                else :
                  yearSimilarity = self.computeYearSimilarity(thisBookID, otherBookID, years)
                  authorSimilarity = self.computeAuthorSimilarity(thisBookID, otherBookID, authors)
                  sim = genreSimilarity * authorSimilarity * yearSimilarity
                  
                self.similarities[thisItem, otherItem] = sim
                self.similarities[otherItem, thisItem] = sim

        print("...done.")

    

    def computeGenreSimilarity(self, book1, book2, genres):
        #computing gener similarities with cosine way
        genres1 = genres[book1]
        genres2 = genres[book2]
        sumxx, sumxy, sumyy = 0, 0, 0
        for i in range(len(genres1)):
            x = genres1[i]
            y = genres2[i]
            sumxx += x * x
            sumyy += y * y
            sumxy += x * y
        
        return sumxy/math.sqrt(sumxx*sumyy)
    
    #compute year similarities .. this way is not good yet
    def computeYearSimilarity(self, book1, book2, years):
        diff = abs(years[book1] - years[book2])
        return math.exp(-0.007 *diff)



    #compute author similarities
    def computeAuthorSimilarity(self,book1, book2 , authors):
      total_similarities = 0
      authors1 = authors[book1]
      authors2 = authors[book2]
      total_elements = len(authors1) + len(authors2)

      for author1 in authors1:
          for author2 in authors2:
              if author1 == author2: # check if strings are exactly the same
                  total_similarities += 1
                  break
      
      similarity_percentage = (total_similarities / total_elements)*2 + 1
      return similarity_percentage

    def exportSimilarities(self):
        np.savetxt('/content/drive/MyDrive/data/CB-Similarities.txt',self.similarities)
          

    def importSimilarities(self):   
      return np.loadtxt('/content/drive/MyDrive/data/CB-Similarities.txt')

    #the most important fonction .. 
    #we calculate the estimate Rating between a user and an item
    #the built in function "predict" will use this function later
    def estimate(self, u, i):

        #making sure that the item and user exsit
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unkown.')
        
        #creating the neighbors list of rated items by the users
        neighbors = []
        for rating in self.trainset.ur[u]:
            Similarity = self.similarities[i,rating[0]]
            neighbors.append( (Similarity, rating[1]) )

        #picking k neighbors
        k_neighbors = heapq.nlargest(self.k, neighbors, key=lambda t: t[0])

        # Compute average sim score of K neighbors weighted by user ratings
        simTotal = weightedSum = 0
        for (simScore, rating) in k_neighbors:
            if (simScore > 0):
                simTotal += simScore
                weightedSum += simScore * rating
            
        if (simTotal == 0):
            raise PredictionImpossible('No neighbors')
            
        predictedRating = weightedSum/simTotal
        return predictedRating
    
    #to predict all estimated ratings of a user
    def EstamedRatingsForUser(self,u):
      prid = []
      for item in range(self.trainset.n_items):     
            rating = self.predict(str(u) , str(self.trainset.to_raw_iid(item)))
            prid.append(rating)

      return prid

SGDAlgorithm class, an SGD-based Algorithm.

In [ ]:
# also extended from an empty recommender template called "AlgoBase"
class SGDAlgorithm(AlgoBase):

    def __init__(self, learning_rate=0.01, n_epochs=20, n_factors=20, reg=0.02):
        #initialize the hyperparameters
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.n_factors = n_factors
        self.reg = reg

    def fit(self, trainset):
        #fitting the trainingset
        AlgoBase.fit(self, trainset)

        # initialize user and item factors , p for users , q for items
        self.users_f = np.random.normal(scale=1.0/self.n_factors, size=(self.trainset.n_users, self.n_factors))
        self.books_f = np.random.normal(scale=1.0/self.n_factors, size=(self.trainset.n_items, self.n_factors))

        # iterate over epochs and optimize factors using SGD
        for _ in range(self.n_epochs):
            for userID, bookID, rating in self.trainset.all_ratings():
                error = rating - np.dot(self.users_f[userID], self.books_f[bookID])
                #correcting the error in user and item factors
                self.users_f[userID] += self.learning_rate*(error*self.books_f[bookID]-self.reg*self.users_f[userID])
                self.books_f[bookID] += self.learning_rate*(error*self.users_f[userID]-self.reg*self.books_f[bookID])
        
    
    #the most important fonction .. 
    #we calculate the estimate Rating between a user and an item
    #the built in function "predict" will use this function later
    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User or item is unknown.')

        #the estimated rating is the dot bettwen item and user learned factors
        est = np.dot(self.users_f[u], self.books_f[i])

        return est
        
    #to predict all estimated ratings of a user
    def EstamedRatingsForUser(self,u):
        prid = []
        for item in range(self.trainset.n_items):     
              rating = self.predict(str(u) , str(self.trainset.to_raw_iid(item)))
              prid.append(rating)

        return prid

HybridAlgorithm class, it uses any algo-base class.

In [ ]:
class HybridAlgorithm:
  def __init__(self, reommendersList):
    self.recommenders = reommendersList

  def fit(self, trainset):
    for algorithm in self.recommenders:
      algorithm.fit(trainset)
  def predict(self, uid, iid, r_ui=None, clip=True, verbose=False):
    predictions = []
    for algorithm in self.recommenders:
      predictions.append(algorithm.predict(uid, iid, r_ui, clip=True, verbose=False))
    mean = 0
    for uid, iid, r_ui, est, details in predictions :
      mean += est
    mean = mean/ len(predictions)
    prediction = [predictions[0][0],predictions[0][1],predictions[0][2], mean ,predictions[0][4]]
    return prediction
  def test(self, testset, verbose=False):
    predictions = [
            self.predict(uid, iid, r_ui_trans, verbose=verbose)
            for (uid, iid, r_ui_trans) in testset ]
    return predictions

Evaluation Data class, the class that provide the necessary data for evaluation.

In [ ]:
class EvaluationData:
    
    def __init__(self, data, popularityRankings):
        
        self.rankings = popularityRankings
        
        #Build a full training set for evaluating overall properties
        self.fullTrainSet = data.build_full_trainset()
        self.fullAntiTestSet = self.fullTrainSet.build_anti_testset()
        
        #Build a 75/25 train/test split for measuring accuracy
        self.trainSet, self.testSet = train_test_split(data, test_size=.25, random_state=1)

        #Using our content based recommender to compute similarty matrix between items so we can measure diversity
        self.simsAlgo = CBAlgorithm()
        print("first, let's calculate similarities bettween books with our CB algorithm, so we can measure diversity later ..")
        self.simsAlgo.fit(self.fullTrainSet)
        
            
    def GetFullTrainSet(self):
        return self.fullTrainSet
    
    def GetFullAntiTestSet(self):
        return self.fullAntiTestSet
    
    def GetTrainSet(self):
        return self.trainSet
    
    def GetTestSet(self):
        return self.testSet   
    
    def GetSimilarities(self):
        return self.simsAlgo
    
    def GetPopularityRankings(self):
        return self.rankings

Recommender Metrics class, the class that contains the evaluation metrics.

In [ ]:
class RecommenderMetrics:

    def MAE(predictions):
        return accuracy.mae(predictions, verbose=False)

    def RMSE(predictions):
        return accuracy.rmse(predictions, verbose=False)

    def additionalMetrices(self ,rankings,TestData,simsAlgo,ratingThreshold=4.0):

      coverage_hits = 0
      usersCount=0
      n_diversity = 0
      total_diversity = 0
      n_novelty = 0
      total_novelty = 0

      for userID in TestData.keys() :
              
              topN = []

              predictions = [self.algorithm.predict(uid, iid, r_ui_trans)
                            for (uid, iid, r_ui_trans) in TestData[userID]]

              for userID, bookID, actualRating, estimatedRating, _ in predictions:
                  topN.append((int(bookID), estimatedRating))

              topN.sort(key=lambda x: x[1], reverse=True) 
              topN = topN[:10]

              # calculating coverage

              hit = False
              if (topN is not []) and (topN [0][1] >= ratingThreshold):
                      hit = True
              if (hit):
                      coverage_hits += 1
              
              #calculating diversty
              pairs = itertools.combinations(topN, 2)
              for pair in pairs:
                      book1 = pair[0][0]
                      book2 = pair[1][0]
                      innerID1 = simsAlgo.trainset.to_inner_iid(str(book1))
                      innerID2 = simsAlgo.trainset.to_inner_iid(str(book2))
                      similarity = simsAlgo.similarities[innerID1,innerID2]
                      total_diversity += similarity
                      n_diversity += 1

              #calculating novelty
              for rating in topN:
                      bookID = rating[0]
                      rank = rankings[bookID]
                      total_novelty += rank
                      n_novelty += 1
              usersCount+=1
              
      coverty = coverage_hits / usersCount
      diversity = (1 - (total_diversity / n_diversity))
      novelty= total_novelty / n_novelty
      return (coverty,diversity,novelty)

Evaluate Algorithm class, the class that implement the evaluation on the algorithm.

In [ ]:
class EvaluatedAlgorithm:
    
    def __init__(self, algorithm, name):
        self.algorithm = algorithm
        self.name = name
        
    def Evaluate(self, evaluationData, n=10, verbose=True):
        metrics = {}
        # Compute accuracy
        if (verbose):
            print("Evaluating accuracy of " + self.name + "  ...")

        self.algorithm.fit(evaluationData.GetTrainSet())
        predictions = self.algorithm.test(evaluationData.GetTestSet())

        metrics["RMSE"] = RecommenderMetrics.RMSE(predictions)
        metrics["MAE"] = RecommenderMetrics.MAE(predictions)

        if (verbose):
              print("Evaluating additional metrics for " + self.name + "  ...")

        self.algorithm.fit(evaluationData.GetFullTrainSet())
        antiTestSet= evaluationData.GetFullAntiTestSet()

        users = defaultdict(list)
        for uid, iid, r_ui_trans in antiTestSet:
          users[uid].append((uid, iid, r_ui_trans))    

        metrics["Coverage"],metrics["Diversity"],metrics["Novelty"]=RecommenderMetrics.additionalMetrices(self
                                                                                                          ,TestData=users
                                                                                                          ,rankings=evaluationData.GetPopularityRankings()
                                                                                                          ,simsAlgo=evaluationData.GetSimilarities())                                         
        if (verbose):
            print("Analysis complete.")

        return metrics
 
    def GetName(self):
        return self.name
    
    def GetAlgorithm(self):
        return self.algorithm

Evaluator class, this class implement the evaluation on many algorithms to compare them.

In [ ]:
class Evaluator:
    
    algorithms = []
    
    def __init__(self, dataset, rankings):
       
        ed = EvaluationData(dataset, rankings)
        print("done...")
        self.dataset = ed
        
    def AddAlgorithm(self, algorithm, name):
        alg = EvaluatedAlgorithm(algorithm, name)
        self.algorithms.append(alg)
        
    def Evaluate(self):
        results = {}
        for algorithm in self.algorithms:
            print("Evaluating ", algorithm.GetName(), "...")
            results[algorithm.GetName()] = algorithm.Evaluate(self.dataset)

        # Print results
        print("{:<10} {:<10} {:<10}{:<10} {:<10} {:<10}".format("Algorithm", "RMSE", "MAE", "Coverage", "Diversity", "Novelty"))
        for (name, metrics) in results.items():
             print("{:<10} {:<10.4f} {:<10.4f}{:<10.4f} {:<10.4f} {:<10.4f}".format(name, metrics["RMSE"], metrics["MAE"],metrics["Coverage"], metrics["Diversity"], metrics["Novelty"]))
                
        print("metrics:\n")
        print("RMSE:      Root Mean Squared Error. Lower values mean better accuracy.")
        print("MAE:       Mean Absolute Error. Lower values mean better accuracy.")
        print("Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.")
        print("Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations")
        print("           for a given user. Higher means more diverse.")
        print("Novelty:   Average popularity rank of recommended items. Higher means more novel.")

You can try the hybrid algorithm or any algo-base algorithm with this code

In [ ]:
def LoadBooksAndRatingsData():
    BD = DataProvider()
    print("Loading book ratings...")
    data = BD.loadData()
    #computing book popularity ranks so we can measure novelty later
    trainset = data.build_full_trainset()
    rankings = BD.getPopularityRanks()
    return (BD, trainset , rankings)

#now lets use it
(BD , data, rankings) = LoadBooksAndRatingsData()

CoRecommender  = CoAlgorithm()

CBRecommender  = CBAlgorithm()

SGDRecommender = SGDAlgorithm()

Recommender = HybridAlgorithm ([CoRecommender,CBRecommender,SGDRecommender])

Recommender.fit(data)
while True:
    user= str(input("please select a user, select 0 to end the programme:"))
    if user == "0":
        break
    try:
        data.to_inner_uid(user)
    except:
        print("invalid user.")
        continue

        #get this user ratings
    userRatings = BD.getUserRatings(int(user))
    loved = []
    hated = []
    ratings_that_exist = []
    for ratings in userRatings:
        #to elemenate this books from recommendations later
        ratings_that_exist.append(ratings[0])

        if (float(ratings[1]) >= 4.0):
            loved.append(ratings)
        elif float(ratings[1]) < 3.0:
            hated.append(ratings)

    print("\nUser ", user , " loved these books:")
    for ratings in loved:
        print(BD.getBookName(int(ratings[0]))+" | with rating :"+ str(ratings[1]) + " | geners :"+str(BD.getStandardGeners()[int(ratings[0])]))
    print("\n...and didn't like these books:")
    for ratings in hated:
        print(BD.getBookName(int(ratings[0]))+" | with rating :"+ str(ratings[1]) + " | geners :"+str(BD.getStandardGeners()[int(ratings[0])]))
    Predictions = Recommender.EstamedRatingsForUser(user)
    #lets convert this data to recommendations
    recommendations = []
    for uid, iid, r_ui, est, details in Predictions :
                    intBookID = int(iid)
                    if intBookID not in ratings_that_exist: # to elemenate books that alerdy readed
                        recommendations.append((intBookID, est))

    #top ratings first
    recommendations.sort(key=lambda x: x[1], reverse=True) 
    print ("\nWe recommend:")     
    for ratings in recommendations[:10]:
        print(BD.getBookName(ratings[0])+ " | Estamed rating : "+ str(ratings[1])+ " | geners :"+str(BD.getStandardGeners()[int(ratings[0])])+ " | year : "+str(BD.getYears()[int(ratings[0])])+" | authors : "+str(BD.getAuthors()[int(ratings[0])]))

You can test the hybrid algorithm or any algo-base algorithm and compare them with this code (this takes a while)

In [ ]:

def LoadBooksAndRatingsData():
    BD = DataProvider()
    print("Loading book ratings...")
    data = BD.loadData()
    #computing book popularity ranks so we can measure novelty later
    rankings = BD.getPopularityRanks()
    return (BD, data , rankings)

#now lets use it
(BD , data  , rankings) = LoadBooksAndRatingsData()
evaluator = Evaluator(data, rankings)

# User-based KNN
CoRecommender  = CoAlgorithm()
evaluator.AddAlgorithm(CoRecommender, "CoRecommender")

# CB-based Algorithm
CBRecommender  = CBAlgorithm()
evaluator.AddAlgorithm(CBRecommender, "CBRecommender")

#our SGD now
SGDRecommender = SGDAlgorithm()
evaluator.AddAlgorithm(SGDRecommender, "SGDRecommender")

CoRecommender  = CoAlgorithm()

CBRecommender  = CBAlgorithm()

SGDRecommender = SGDAlgorithm()

HybridRecommender = HybridAlgorithm ([CoRecommender,CBRecommender,SGDRecommender])

evaluator.AddAlgorithm(HybridRecommender, "HybridRecommender")

#standar NMF now
standardNMF = NMF(n_epochs=20,n_factors=20)
evaluator.AddAlgorithm(standardNMF, "NMF")

# KNNBasic
standardClustering = KNNBasic()
evaluator.AddAlgorithm(standardClustering, "KNNBasic")

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

# Test !
evaluator.Evaluate()

first, let's calculate similarities bettween books with our CB algorithm, so we can measure diversity later ..
Computing content-based similarity matrix...
0  of  9736
100  of  9736
200  of  9736
300  of  9736
400  of  9736
500  of  9736
600  of  9736
700  of  9736
800  of  9736
900  of  9736
1000  of  9736
1100  of  9736
1200  of  9736
1300  of  9736
1400  of  9736
1500  of  9736
1600  of  9736
1700  of  9736
1800  of  9736
1900  of  9736
2000  of  9736
2100  of  9736
2200  of  9736
2300  of  9736
2400  of  9736
2500  of  9736
2600  of  9736
2700  of  9736
2800  of  9736
2900  of  9736
3000  of  9736
3100  of  9736
3200  of  9736
3300  of  9736
3400  of  9736
3500  of  9736
3600  of  9736
3700  of  9736
3800  of  9736
3900  of  9736
4000  of  9736
4100  of  9736
4200  of  9736
4300  of  9736
4400  of  9736
4500  of  9736
4600  of  9736
4700  of  9736
4800  of  9736
4900  of  9736
5000  of  9736
5100  of  9736
5200  of  9736
5300  of  9736
5400  of  9736
5500  of  9736
5600  of  9736
5